In [1]:
import numpy as np
import matplotlib.pyplot as plt
import desc.io
from desc.grid import LinearGrid, ConcentricGrid
from desc.objectives import (
    ObjectiveFunction,
    FixBoundaryR,
    FixBoundaryZ,
    FixPressure,
    FixCurrent,
    FixIota,
    FixPsi,
    AspectRatio,
    ForceBalance,
    QuasisymmetryTwoTerm,
    RotationalTransform
)
from desc.optimize import Optimizer
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary,
    plot_3d
)

from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.profiles import PowerSeriesProfile

DESC version 0+untagged.9463.gc6b5ccf.dirty,using JAX backend, jax version=0.4.38, jaxlib version=0.4.38, dtype=float64
Using device: CPU, with 23.73 GB available memory


In [2]:
eq_fam = desc.io.load("inputs/nfp2_QA.h5")
eq_init = eq_fam[-1]
eq_qs = eq_init.copy()

In [3]:
optimizer = Optimizer("lsq-exact")

grid_vol = LinearGrid(
    M=eq_qs.M_grid,
    N=eq_qs.N_grid,
    NFP=eq_qs.NFP,
    rho=np.arange(0.1,1.1,0.1),
    sym=eq_qs.sym
)

In [4]:
for maxMode in range(1,2):
    fixed_R_modes = np.vstack(([0,0,0], eq_qs.surface.R_basis.modes[np.max(np.abs(eq_qs.surface.R_basis.modes), 1) > maxMode, :]))
    free_R_modes = eq_qs.surface.R_basis.modes[np.max(np.abs(eq_qs.surface.R_basis.modes), 1) <= maxMode, :]
    free_R_modes = free_R_modes[np.max(np.abs(free_R_modes), 1) > 0, :]
    
    fixed_Z_modes = eq_qs.surface.Z_basis.modes[np.max(np.abs(eq_qs.surface.Z_basis.modes), 1) > maxMode, :]
    free_Z_modes = eq_qs.surface.Z_basis.modes[np.max(np.abs(eq_qs.surface.Z_basis.modes), 1) <= maxMode, :]
    constWeight = 10
    
    objective_f = ObjectiveFunction((
        AspectRatio(eq=eq_qs, target=6, weight=1)
    ))

    numRbounds = len(free_R_modes)
    numZbounds = len(free_Z_modes)
    rub = np.ones(numRbounds)
    zub = np.ones(numZbounds)
    Rbounds = (-rub, rub)
    Zbounds = (-zub, zub)
    
    constraints = (
        FixBoundaryR(eq=eq_qs, modes=fixed_R_modes),
        FixBoundaryZ(eq=eq_qs, modes=fixed_Z_modes),
        FixPressure(eq=eq_qs),  # fix pressure profile
        FixPsi(eq=eq_qs),
        FixCurrent(eq=eq_qs),
    )    
    eq_qs, result = eq_qs.optimize(
        objective=objective_f,
        constraints=constraints,
        optimizer=optimizer,
        copy=True,
        x_scale = np.ones(106),
        verbose=2,
        maxiter=20,
        options={}
    )


/Users/aak572/mason_desc_branch/DESC/desc/optimize/optimizer.py:474: FutureWarning: 
                Nonlinear constraints detected but method lsq-exact does not support
                nonlinear constraints. Defaulting to method "proximal-lsq-exact"
                In the future this will raise an error. To ignore this warning, specify
                a wrapper "proximal-" to convert the nonlinearly constrained problem
                into an unconstrained one.
                
  warnings.warn(


Building objective: aspect ratio
Precomputing transforms
Timer: Precomputing transforms = 443 ms
Timer: Objective build = 488 ms
Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 493 ms
Timer: Objective build = 2.41 sec
Timer: Proximal projection build = 8.41 sec
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Timer: Objective build = 1.01 sec
Timer: Linear constraint projection build = 663 ms


/Users/aak572/mason_desc_branch/DESC/desc/optimize/optimizer.py:261: UserWarning: method lsq-exact is not intended for scalar objective function
  warnings.warn(


Number of parameters: 0
Number of objectives: 1
Starting optimization
Using method: lsq-exact


ValueError: zero-size array to reduction operation max which has no identity

In [ ]:
print(result.x.shape, result.allx[-1].shape)
print(np.max(np.abs(result.allx[-1] - result.allx[0])))
print(np.mean(np.abs(result.allx[-1] - result.allx[0])))
print(objective_f.compute_scalar(result.allx[-1]))
print(result.allx[-1][np.abs(result.allx[-1]) > 1e-6])
x_temp = np.array(result.allx[-1].copy())
x_temp[np.abs(x_temp) < 1e-6] = 0.0
# x_temp[:120] = 0.0
print(x_temp)
print(objective_f.compute_scalar(x_temp))

In [ ]:
plot_boundary(eq_qs, figsize=(6,6))
plot_boozer_surface(eq_qs, figsize=(6,6))
plot_3d(eq_qs, "|B|", figsize=(6,6))

In [ ]:
eq_qs = eq_init.copy()
objective_f = ObjectiveFunction((AspectRatio(eq=eq_qs, target=6, weight=1)))

optimizer = Optimizer("Turbo1")

for maxMode in range(1,2):
    fixed_R_modes = np.vstack(([0,0,0], eq_qs.surface.R_basis.modes[np.max(np.abs(eq_qs.surface.R_basis.modes), 1) > maxMode, :]))
    free_R_modes = eq_qs.surface.R_basis.modes[np.max(np.abs(eq_qs.surface.R_basis.modes), 1) <= maxMode, :]
    free_R_modes = free_R_modes[np.max(np.abs(free_R_modes), 1) > 0, :]
    
    fixed_Z_modes = eq_qs.surface.Z_basis.modes[np.max(np.abs(eq_qs.surface.Z_basis.modes), 1) > maxMode, :]
    free_Z_modes = eq_qs.surface.Z_basis.modes[np.max(np.abs(eq_qs.surface.Z_basis.modes), 1) <= maxMode, :]
    constWeight = 10
    numRbounds = len(free_R_modes)
    numZbounds = len(free_Z_modes)
    rub = np.ones(numRbounds)
    zub = np.ones(numZbounds)
    Rbounds = (-rub, rub)
    Zbounds = (-zub, zub)
    
    constraints = (
        FixBoundaryR(eq=eq_qs, modes=fixed_R_modes),
        FixBoundaryZ(eq=eq_qs, modes=fixed_Z_modes),
        FixPressure(eq=eq_qs),  # fix pressure profile
        FixPsi(eq=eq_qs),
        FixCurrent(eq=eq_qs),
    )    
    global_eq_qs, result = eq_qs.optimize(
        objective=objective_f,
        # constraints=constraints,
        optimizer=optimizer,
        copy=True,
        x_scale = np.ones(106),
        verbose=2,
        maxiter=10000,
        options={"trust_regions":1, "box_size":1e-5, "training_steps": 500, "batch_size":200}
    )

In [ ]:
print(objective_f.compute_scalar(result.allx[np.argmin(result.allfun)]))
print(np.shape(result.x), np.shape(result.allx[-1]), np.shape(result.allx))
print(result.x)

In [ ]:
plot_boundary(global_eq_qs, figsize=(6,6))
plot_boozer_surface(global_eq_qs, figsize=(6,6))
plot_3d(global_eq_qs, "|B|", figsize=(6,6))

In [ ]:
objective_f.print_value(result.allx[np.argmin(result.allfun)])

In [ ]:
plot_boundary(eq_init, figsize=(6,6))
plot_boozer_surface(eq_init, figsize=(6,6))
plot_3d(eq_init, "|B|", figsize=(6,6))